# Real model

In [1]:
pip install ortools


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install folium


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import matplotlib.colors as mcolors


In [4]:
repo_url = 'https://raw.githubusercontent.com/baertsch/MGT-530-SLO/main/'

In [5]:
vehicle_matrix = pd.read_excel(repo_url + 'vhc_matrix_city_excluded.xlsx')
vehicle_matrix = vehicle_matrix.iloc[0:, :1]
vehicle_matrix

,Places
0,16
1,91
2,50
3,20
4,127
5,100


In [6]:
v = vehicle_matrix.values.tolist()
v = [item[0] for item in v]

In [7]:
full_data = pd.read_csv(repo_url + 'full_data.csv')

In [8]:
d_venoge = full_data[['NPA', 'distance_to_venoge_km']]

### Code to display the map

In [9]:
npa_coords = full_data[['NPA', 'lat', 'lng']]
depot_row = pd.DataFrame([{'NPA': 0, 'lat': 46.60396069250175, 'lng': 6.538034286509177}])
npa_coords = pd.concat([npa_coords, depot_row], ignore_index=True)
npa_to_coords = npa_coords.set_index('NPA')[['lat', 'lng']].to_dict('index')

In [10]:
def get_color_palette(n):
    palette = sns.color_palette("pastel", n)
    return [mcolors.rgb2hex(color) for color in palette]

def plot_routes_folium(df_results, npa_to_coords):
    depot_coords = npa_to_coords[0]
    m = folium.Map(location=[depot_coords['lat'], depot_coords['lng']], zoom_start=11)
    num_vehicles = df_results[df_results['vehicle_id'] != 'Total'].shape[0]
    colors = get_color_palette(num_vehicles)

    for idx, row in df_results.iterrows():
        if row['vehicle_id'] == 'Total':
            continue
        fg = folium.FeatureGroup(name=f"Vehicle {row['vehicle_id']}")
        route = row['route']
        points = []
        for npa in route:
            try:
                npa_int = int(npa)
            except Exception:
                continue
            coords = npa_to_coords.get(npa_int)
            if coords:
                points.append((coords['lat'], coords['lng']))
        if points:
            folium.PolyLine(
                points,
                color=colors[idx % len(colors)],
                weight=5,
                opacity=1,  # Fully opaque
                popup=f"Vehicle ID: {row['vehicle_id']}"
            ).add_to(fg)
            for lat, lng in points:
                folium.CircleMarker([lat, lng], radius=3, color='black').add_to(fg)
        fg.add_to(m)
    folium.LayerControl(collapsed=False).add_to(m)
    return m

## Wednesday

In [11]:
demand_wed = pd.read_csv(repo_url + 'Tickets_du_Mercredi.csv',header=None)

### Distance matrix

In [12]:
distance_matrix_wed = pd.read_csv(repo_url + 'data/dm_mercredi.csv',header=None)
distance_matrix_wed = distance_matrix_wed.iloc[1:,1:]

In [13]:
df = demand_wed.iloc[1:, 2:3]
df = df.astype('int64')
df = pd.merge(d_venoge, df, left_on='NPA', right_on=2)
df = df.iloc[:, 1:2]
depot_distances = df.iloc[:, 0].tolist()

# Insert depot as first column
distance_matrix_wed = distance_matrix_wed.copy()
distance_matrix_wed.insert(0, 'Depot', depot_distances)

# Insert depot as first row (must match new number of columns)
depot_row = [0.0] + depot_distances
distance_matrix_wed.loc[-1] = depot_row
distance_matrix_wed.index = distance_matrix_wed.index + 1
distance_matrix_wed = distance_matrix_wed.sort_index()

# Reset column names and index to integers
distance_matrix_wed.columns = range(distance_matrix_wed.shape[1])
distance_matrix_wed.index = range(distance_matrix_wed.shape[0])

In [14]:
dm_wed = distance_matrix_wed.values.tolist()
for i in range(len(dm_wed)):
    dm_wed[i][i] = 0
dm_wed = [[9999 if (isinstance(x, float) and np.isinf(x)) else x for x in row] for row in dm_wed]
dm_wed = [[int(float(x)) for x in row] for row in dm_wed]


### Demand matrix

In [15]:
demand_wed = demand_wed.iloc[1:, 4:]

In [16]:
d_wed = demand_wed.values.tolist()
d_wed = [int(float(item[0])) for item in d_wed]
d_wed = [0] + d_wed

### Vehicle Matrix

In [17]:
v_wed_full = v * len(dm_wed)

### CVRP

In [18]:
npa_list = pd.read_csv(repo_url + 'Tickets_du_Mercredi.csv', header=None).iloc[1:, 2].reset_index(drop=True).tolist()
node_to_npa = [0] + npa_list  # index 0 is depot

In [19]:
def create_data_model(subset_distance_matrix, subset_demands, capacities):
    """Stores the data for the problem."""
    data = {}
    # Data multiplied by a factor of 10 to avoid non-integer numbers
    data['distance_matrix'] = subset_distance_matrix
    data['demands'] = subset_demands
    data['vehicle_capacities'] = capacities
    data['num_vehicles'] = len(capacities)
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Collects solution in a DataFrame for easy CSV export, with NPA in route."""
    results = []
    total_distance = 0
    total_load = 0
    unused_vehicles = []

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_distance = 0
        route_load = 0
        route = []
        capacity = data['vehicle_capacities'][vehicle_id]

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            demand = data['demands'][node_index]
            route_load += demand
            route.append(node_to_npa[node_index])
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        end_node = manager.IndexToNode(index)
        route.append(node_to_npa[end_node])

        if route_load == 0:
            unused_vehicles.append(vehicle_id)
            continue  # Skip empty routes

        results.append({
            "vehicle_id": vehicle_id,
            "capacity": capacity,
            "route": route,
            "distance_km": route_distance / 10,
            "load": route_load,
        })

        total_distance += route_distance
        total_load += route_load

    # Add summary row
    results.append({
        "vehicle_id": "Total",
        "capacity": "",
        "route": "",
        "distance_km": total_distance / 10,
        "load": total_load,
    })

    df_results = pd.DataFrame(results)
    print(df_results)
    return df_results

def main():
    """Solve the CVRP problem."""

    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix= dm_wed, subset_demands=d_wed, capacities=v_wed_full)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        df_results= print_solution(data, manager, routing, solution)
        m = plot_routes_folium(df_results, npa_to_coords)
        display(m)


main()

    vehicle_id capacity                                 route  distance_km  \
0            1       91                          [0, 1032, 0]          1.8   
1         1660      127  [0, 1299, 1296, 1295, 1291, 1023, 0]         11.0   
2         1661      100                          [0, 1304, 0]          0.8   
3         1666      127              [0, 1275, 1297, 1004, 0]         12.4   
4         1667      100                          [0, 1304, 0]          0.8   
..         ...      ...                                   ...          ...   
151       1981       91                    [0, 1034, 1376, 0]          2.1   
152       1982       50                          [0, 1305, 0]          0.4   
153       1984      127              [0, 1062, 1063, 1041, 0]          4.4   
154       1985      100                          [0, 1304, 0]          0.8   
155      Total                                                       603.0   

      load  
0       77  
1      127  
2      100  
3      125 

## Thursday

In [20]:
demand_thur = pd.read_csv(repo_url + 'Tickets_du_Jeudi.csv',header=None)

### Distance Matrix

In [21]:
distance_matrix_thur = pd.read_csv(repo_url + 'data/dm_jeudi.csv',header=None)
distance_matrix_thur = distance_matrix_thur.iloc[1:,1:]

In [22]:
df = demand_thur.iloc[1:, 2:3]
df = df.astype('int64')
df = pd.merge(d_venoge, df, left_on='NPA', right_on=2)
df = df.iloc[:, 1:2]
depot_distances = df.iloc[:, 0].tolist()

# Insert depot as first column
distance_matrix_thur = distance_matrix_thur.copy()
distance_matrix_thur.insert(0, 'Depot', depot_distances)

# Insert depot as first row (must match new number of columns)
depot_row = [0.0] + depot_distances
distance_matrix_thur.loc[-1] = depot_row
distance_matrix_thur.index = distance_matrix_thur.index + 1
distance_matrix_thur = distance_matrix_thur.sort_index()

# Reset column names and index to integers
distance_matrix_thur.columns = range(distance_matrix_thur.shape[1])
distance_matrix_thur.index = range(distance_matrix_thur.shape[0])

In [23]:
dm_thur = distance_matrix_thur.values.tolist()
for i in range(len(dm_thur)):
    dm_thur[i][i] = 0
dm_thur = [[9999 if (isinstance(x, float) and np.isinf(x)) else x for x in row] for row in dm_thur]
dm_thur = [[int(float(x)) for x in row] for row in dm_thur]

### Demand Matrix

In [24]:
demand_thur = demand_thur.iloc[1:, 4:]

In [25]:
d_thur = demand_thur.values.tolist()
d_thur = [int(float(item[0])) for item in d_thur]
d_thur = [0] + d_thur

### Vehicule Matrix

In [26]:
v_thur_full = v * len(dm_thur)

### CVRP

In [27]:
npa_list = pd.read_csv(repo_url + 'Tickets_du_Jeudi.csv', header=None).iloc[1:, 2].reset_index(drop=True).tolist()
node_to_npa = [0] + npa_list  # index 0 is depot

In [28]:
def main():
    """Solve the CVRP problem."""

    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix= dm_thur, subset_demands=d_thur, capacities=v_thur_full)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        df_results= print_solution(data, manager, routing, solution)
        m = plot_routes_folium(df_results, npa_to_coords)
        display(m)


main()

    vehicle_id capacity                                             route  \
0            1       91                                [0, 1303, 1035, 0]   
1            4      127                    [0, 1061, 1515, 1514, 1052, 0]   
2           10      127                                      [0, 1304, 0]   
3         1468      127  [0, 1846, 1882, 1885, 1820, 1822, 1817, 1018, 0]   
4         1469      100                                      [0, 1304, 0]   
..         ...      ...                                               ...   
110       1663       91                                [0, 1306, 1377, 0]   
111       1664       50              [0, 1307, 1352, 1354, 1439, 1313, 0]   
112       1666      127                          [0, 1443, 1063, 1033, 0]   
113       1667      100                                      [0, 1304, 0]   
114      Total                                                              

     distance_km   load  
0            0.5     88  
1            7.1    125

## Friday

In [29]:
demand_fri = pd.read_csv(repo_url + 'Tickets_du_Vendredi.csv',header=None)

### Distance Matrix

In [30]:
distance_matrix_fri = pd.read_csv(repo_url + 'data/dm_vendredi.csv',header=None)
distance_matrix_fri = distance_matrix_fri.iloc[1:,1:]

In [31]:
df = demand_fri.iloc[1:, 2:3]
df = df.astype('int64')
df = pd.merge(d_venoge, df, left_on='NPA', right_on=2)
df = df.iloc[:, 1:2]
depot_distances = df.iloc[:, 0].tolist()

# Insert depot as first column
distance_matrix_fri = distance_matrix_fri.copy()
distance_matrix_fri.insert(0, 'Depot', depot_distances)

# Insert depot as first row (must match new number of columns)
depot_row = [0.0] + depot_distances
distance_matrix_fri.loc[-1] = depot_row
distance_matrix_fri.index = distance_matrix_fri.index + 1
distance_matrix_fri = distance_matrix_fri.sort_index()

# Reset column names and index to integers
distance_matrix_fri.columns = range(distance_matrix_fri.shape[1])
distance_matrix_fri.index = range(distance_matrix_fri.shape[0])

In [32]:
dm_fri = distance_matrix_fri.values.tolist()
for i in range(len(dm_fri)):
    dm_fri[i][i] = 0
dm_fri = [[9999 if (isinstance(x, float) and np.isinf(x)) else x for x in row] for row in dm_fri]
dm_fri = [[int(float(x)) for x in row] for row in dm_fri]

## Demand Matrix

In [33]:
demand_fri = demand_fri.iloc[1:, 4:]

In [34]:
d_fri = demand_fri.values.tolist()
d_fri = [int(float(item[0])) for item in d_fri]
d_fri = [0] + d_fri

### Vehicule Matrix

In [35]:
v_fri_full = v * len(dm_fri)

### CVRP

In [36]:
npa_list = pd.read_csv(repo_url + 'Tickets_du_Vendredi.csv', header=None).iloc[1:, 2].reset_index(drop=True).tolist()
node_to_npa = [0] + npa_list  # index 0 is depot

In [37]:
def main():
    """Solve the CVRP problem."""

    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix= dm_fri, subset_demands=d_fri, capacities=v_fri_full)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        df_results= print_solution(data, manager, routing, solution)
        m = plot_routes_folium(df_results, npa_to_coords)
        display(m)



main()

   vehicle_id capacity                                              route  \
0           0       16                                       [0, 1072, 0]   
1           1       91  [0, 1412, 1404, 1462, 1463, 1545, 1589, 1586, ...   
2        1295      100                                       [0, 1304, 0]   
3        1300      127                           [0, 1588, 1083, 1033, 0]   
4        1301      100                                       [0, 1304, 0]   
..        ...      ...                                                ...   
80       1435       91  [0, 1184, 1274, 1262, 1296, 1295, 1290, 1263, ...   
81       1436       50                                       [0, 1306, 0]   
82       1438      127                                 [0, 1303, 1031, 0]   
83       1439      100                                       [0, 1304, 0]   
84      Total                                                               

    distance_km  load  
0           5.0    13  
1          14.8    91  
2  

## Saturday

In [38]:
demand_sat = pd.read_csv(repo_url + 'Tickets_du_Samedi.csv',header=None)

### Distance Matrix

In [39]:
distance_matrix_sat = pd.read_csv(repo_url + 'data/dm_samedi.csv',header=None)
distance_matrix_sat = distance_matrix_sat.iloc[1:,1:]

In [40]:
df = demand_sat.iloc[1:, 2:3]
df = df.astype('int64')
df = pd.merge(d_venoge, df, left_on='NPA', right_on=2)
df = df.iloc[:, 1:2]
depot_distances = df.iloc[:, 0].tolist()

# Insert depot as first column
distance_matrix_sat = distance_matrix_sat.copy()
distance_matrix_sat.insert(0, 'Depot', depot_distances)

# Insert depot as first row (must match new number of columns)
depot_row = [0.0] + depot_distances
distance_matrix_sat.loc[-1] = depot_row
distance_matrix_sat.index = distance_matrix_sat.index + 1
distance_matrix_sat = distance_matrix_sat.sort_index()

# Reset column names and index to integers
distance_matrix_sat.columns = range(distance_matrix_sat.shape[1])
distance_matrix_sat.index = range(distance_matrix_sat.shape[0])

In [41]:
dm_sat = distance_matrix_sat.values.tolist()
for i in range(len(dm_sat)):
    dm_sat[i][i] = 0
dm_sat = [[9999 if (isinstance(x, float) and np.isinf(x)) else x for x in row] for row in dm_sat]
dm_sat = [[int(float(x)) for x in row] for row in dm_sat]

### Demand Matrix

In [42]:
demand_sat = demand_sat.iloc[1:, 4:]

In [43]:
d_sat = demand_sat.values.tolist()
d_sat = [int(float(item[0])) for item in d_sat]
d_sat = [0] + d_sat

### Vehicule Matrix

In [44]:
v_sat_full = v * len(dm_sat)

### CVRP

In [45]:
npa_list = pd.read_csv(repo_url + 'Tickets_du_Samedi.csv', header=None).iloc[1:, 2].reset_index(drop=True).tolist()
node_to_npa = [0] + npa_list  # index 0 is depot

In [46]:
def main():
    """Solve the CVRP problem."""

    # Instantiate the data problem.
    data = create_data_model(subset_distance_matrix= dm_sat, subset_demands=d_sat, capacities=v_sat_full)
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')



    # Complete here
    seach_parameters = pywrapcp.DefaultRoutingSearchParameters()
    seach_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
    
    solution = routing.SolveWithParameters(seach_parameters)

    if solution:
        df_results= print_solution(data, manager, routing, solution)
        m = plot_routes_folium(df_results, npa_to_coords)
        display(m)

main()

   vehicle_id capacity                           route  distance_km  load
0           1       91  [0, 1010, 1083, 1078, 1068, 0]          5.8    89
1           4      127  [0, 1033, 1416, 1407, 1406, 0]          6.1   124
2           5      100                    [0, 1304, 0]          0.8   100
3        1319      100                    [0, 1304, 0]          0.8   100
4        1324      127  [0, 1815, 1817, 1816, 1006, 0]          7.9   124
..        ...      ...                             ...          ...   ...
93       1489       91                    [0, 1305, 0]          0.4    91
94       1490       50                    [0, 1022, 0]          2.2    40
95       1492      127  [0, 1514, 1510, 1063, 1033, 0]          6.3   127
96       1493      100                    [0, 1303, 0]          0.0    92
97      Total                                                 451.1  9075

[98 rows x 5 columns]
